## Lab 3
### Part 2: Dealing with overfitting

Today we work with [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) (*hint: it is available in `torchvision`*).

Your goal for today:
1. Train a FC (fully-connected) network that achieves >= 0.885 test accuracy.
2. Cause considerable overfitting by modifying the network (e.g. increasing the number of network parameters and/or layers) and demonstrate in in the appropriate way (e.g. plot loss and accurasy on train and validation set w.r.t. network complexity).
3. Try to deal with overfitting (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results.

__Please, write a small report describing your ideas, tries and achieved results in the end of this file.__

*Note*: Tasks 2 and 3 are interrelated, in task 3 your goal is to make the network from task 2 less prone to overfitting. Task 1 is independent from 2 and 3.

*Note 2*: We recomment to use Google Colab or other machine with GPU acceleration.

In [20]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchsummary
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os

from tqdm import tqdm
from plotly import express as ex
import plotly.graph_objects as go

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [3]:
# Technical function
def mkdir(path):
    if not os.path.exists(root_path):
        os.mkdir(root_path)
        print('Directory', path, 'is created!')
    else:
        print('Directory', path, 'already exists!')
        
root_path = 'fmnist'
mkdir(root_path)

Directory fmnist already exists!


In [4]:
download = True
train_transform = transforms.ToTensor()
test_transform = transforms.ToTensor()
transforms.Compose((transforms.ToTensor()))


fmnist_dataset_train = torchvision.datasets.FashionMNIST(root_path, 
                                                        train=True, 
                                                        transform=train_transform,
                                                        target_transform=None,
                                                        download=download)
fmnist_dataset_test = torchvision.datasets.FashionMNIST(root_path, 
                                                       train=False, 
                                                       transform=test_transform,
                                                       target_transform=None,
                                                       download=download)

In [5]:
train_loader = torch.utils.data.DataLoader(fmnist_dataset_train, 
                                           batch_size=256,
                                           shuffle=True,
                                           num_workers=2,)
test_loader = torch.utils.data.DataLoader(fmnist_dataset_test,
                                          batch_size=256,
                                          shuffle=False,
                                          num_workers=2)

In [6]:
len(fmnist_dataset_test)

10000

In [7]:
for img, label in train_loader:
    print(img.shape)
#     print(img)
    print(label.shape)
    print(label.size(0))
    break

torch.Size([256, 1, 28, 28])
torch.Size([256])
256


### Task 1
Train a network that achieves $\geq 0.885$ test accuracy. It's fine to use only Linear (`nn.Linear`) layers and activations/dropout/batchnorm. Convolutional layers might be a great use, but we will meet them a bit later.

In [76]:
class TinyNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            nn.Linear(input_shape, input_shape),
            nn.ReLU(),
            nn.Linear(input_shape, num_classes),
            nn.Softmax()
        )
        
    def forward(self, inp):  
        out = self.model(inp)
        return out

In [77]:
torchsummary.summary(TinyNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 784]         615,440
              ReLU-3                  [-1, 784]               0
            Linear-4                   [-1, 10]           7,850
           Softmax-5                   [-1, 10]               0
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.38
Estimated Total Size (MB): 2.40
----------------------------------------------------------------


Your experiments come here:

In [78]:
model = TinyNeuralNetwork().to(device)

learning_rate = 0.001
num_epochs = 50

opt = torch.optim.Adam(model.parameters(), lr=learning_rate) # YOUR CODE HERE
loss_func =  nn.CrossEntropyLoss()  # YOUR CODE HERE


# Your experiments, training and validation loops here

In [79]:
# Train the model
total_step = len(train_loader)
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):  
        
        # Move tensors to the configured device
#         images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        # Backprpagation and optimization
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    #Compute Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
#         print('Accuracy on test images: {} %'.format()
        print ('Epoch [{}/{}], \t Accuracy {}%'.format(epoch+1, num_epochs, accuracy ))

  0%|                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Step [100/235], Loss: 1.7053
Epoch [1/50], Step [200/235], Loss: 1.6837


  2%|███                                                                                                                                                     | 1/50 [00:05<04:20,  5.32s/it]

Epoch [1/50], 	 Accuracy 79.01%
Epoch [2/50], Step [100/235], Loss: 1.6556
Epoch [2/50], Step [200/235], Loss: 1.6107


  4%|██████                                                                                                                                                  | 2/50 [00:10<04:14,  5.31s/it]

...

Epoch [48/50], 	 Accuracy 88.8%
Epoch [49/50], Step [100/235], Loss: 1.5322
Epoch [49/50], Step [200/235], Loss: 1.5367


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 49/50 [05:11<00:06,  6.80s/it]

Epoch [49/50], 	 Accuracy 89.08%
Epoch [50/50], Step [100/235], Loss: 1.5272
Epoch [50/50], Step [200/235], Loss: 1.5372


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:18<00:00,  6.37s/it]

Epoch [50/50], 	 Accuracy 88.96%


In [80]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
#         print('Accuracy on test images: {} %'.format()
    print ('Accuracy {}%'.format( accuracy ))

Accuracy 88.96%


### Task 2: Overfit it.
Build a network that will overfit to this dataset. Demonstrate the overfitting in the appropriate way (e.g. plot loss and accurasy on train and test set w.r.t. network complexity).

*Note:* you also might decrease the size of `train` dataset to enforce the overfitting and speed up the computations.

In [158]:
# train_loader = torch.utils.data.DataLoader(fmnist_dataset_train, 
#                                            batch_size=256,
#                                            shuffle=True,
#                                            num_workers=2,)

In [159]:
class OverfittingNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            nn.Linear(input_shape, input_shape),
            nn.ReLU(),
            nn.Linear(input_shape, num_classes),
            nn.Softmax()
        )
        
    def forward(self, inp):       
        out = self.model(inp)
        return out

In [160]:
torchsummary.summary(OverfittingNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 784]         615,440
              ReLU-3                  [-1, 784]               0
            Linear-4                   [-1, 10]           7,850
           Softmax-5                   [-1, 10]               0
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.38
Estimated Total Size (MB): 2.40
----------------------------------------------------------------


In [161]:
model = OverfittingNeuralNetwork().to(device)

learning_rate = 0.001
num_epochs = 100

opt = torch.optim.Adam(model.parameters(), lr=learning_rate) # YOUR CODE HERE
loss_func =  nn.CrossEntropyLoss()  # YOUR CODE HERE

# Your experiments, come here

In [162]:
train_loader = torch.utils.data.DataLoader(fmnist_dataset_train, 
                                           batch_size=256,
                                           shuffle=False,
                                           num_workers=2,)
test_loader = torch.utils.data.DataLoader(fmnist_dataset_test,
                                          batch_size=256,
                                          shuffle=False,
                                          num_workers=2)

In [163]:
print(len(train_loader))
print(len(test_loader))

train_dataset_stop_index = 1 #with batch size 256 (part ~ 35%)

235
40


In [164]:
# Train the model
total_step = len(train_loader)
accuracy_train_list = []
accuracy_test_list = []
loss_train_list = []
loss_test_list = []

for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):  
        if i == train_dataset_stop_index :
            break
            
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        # Backprpagation and optimization
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        if (i+1) % 39 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, train_dataset_stop_index, loss.item()))
    
    #Compute train Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):  
            if i == train_dataset_stop_index :
                break
            
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy_train = 100 * correct / total
        accuracy_train_list.append(accuracy_train)
        
        loss_train_list.append(loss.item())
    
    
    #Compute test Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy_test = 100 * correct / total
        accuracy_test_list.append(accuracy_test)
        
        loss_test_list.append(loss.item())
    
    print ('Epoch [{}/{}], \t Train accuracy {}%'.format(epoch+1, num_epochs, accuracy_train ))
    print ('Epoch [{}/{}], \t Test accuracy {}%'.format(epoch+1, num_epochs, accuracy_test ))



  0%|                                                                                                                                                               | 0/100 [00:00<?, ?it/s]

  1%|█▌                                                                                                                                                     | 1/100 [00:04<07:06,  4.31s/it]

Epoch [1/100], 	 Train accuracy 53.125%
Epoch [1/100], 	 Test accuracy 48.09%




  2%|███                                                                                                                                                    | 2/100 [00:09<07:23,  4.52s/it]

Epoch [2/100], 	 Train accuracy 51.953125%
Epoch [2/100], 	 Test accuracy 48.92%




  3%|████▌                                                                                                                                                  | 3/100 [00:14<07:29,  4.63s/it]

...

Epoch [98/100], 	 Train accuracy 96.875%
Epoch [98/100], 	 Test accuracy 76.37%




 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 99/100 [08:34<00:04,  4.95s/it]

Epoch [99/100], 	 Train accuracy 96.875%
Epoch [99/100], 	 Test accuracy 76.46%




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:38<00:00,  5.19s/it]

Epoch [100/100], 	 Train accuracy 96.875%
Epoch [100/100], 	 Test accuracy 76.43%


In [165]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= list(range(len(accuracy_train_list))),
    y=accuracy_train_list,
))
fig.add_trace(go.Scatter(
    x= list(range(len(accuracy_test_list))),
    y=accuracy_test_list,

))

fig.show()

In [166]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= list(range(len(loss_train_list))),
    y=loss_train_list,
))
fig.add_trace(go.Scatter(
    x= list(range(len(loss_test_list))),
    y=loss_test_list,

))

fig.show()

### Task 3: Fix it.
Fix the overfitted network from the previous step (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results. 

In [174]:
class FixedNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            
            nn.Linear(input_shape, input_shape),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(input_shape, num_classes),
            nn.Softmax()
        )
        
    def forward(self, inp):       
        out = self.model(inp)
        return out

In [175]:
torchsummary.summary(FixedNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 784]         615,440
              ReLU-3                  [-1, 784]               0
           Dropout-4                  [-1, 784]               0
            Linear-5                   [-1, 10]           7,850
           Softmax-6                   [-1, 10]               0
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.38
Estimated Total Size (MB): 2.40
----------------------------------------------------------------


In [191]:
model = FixedNeuralNetwork().to(device)

learning_rate = 0.001
num_epochs = 50

opt = torch.optim.Adam(model.parameters(), lr=learning_rate) # YOUR CODE HERE
loss_func =  nn.CrossEntropyLoss()  # YOUR CODE HERE

# Your experiments, come here

In [192]:
train_loader = torch.utils.data.DataLoader(fmnist_dataset_train, 
                                           batch_size=2048,
                                           shuffle=False,
                                           num_workers=2,)
test_loader = torch.utils.data.DataLoader(fmnist_dataset_test,
                                          batch_size=256,
                                          shuffle=False,
                                          num_workers=2)

In [193]:
print(len(train_loader))
print(len(test_loader))

train_dataset_stop_index = 300 #with batch size 256 (part ~ 35%)

30
40


In [194]:
# Train the model
total_step = len(train_loader)
accuracy_train_list = []
accuracy_test_list = []
loss_train_list = []
loss_test_list = []

for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):  
        if i == train_dataset_stop_index :
            break
            
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        # Backprpagation and optimization
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, train_dataset_stop_index, loss.item()))
    
    #Compute train Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):  
            if i == train_dataset_stop_index :
                break
            
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy_train = 100 * correct / total
        accuracy_train_list.append(accuracy_train)
        
        loss_train_list.append(loss.item())
    
    
    #Compute test Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy_test = 100 * correct / total
        accuracy_test_list.append(accuracy_test)
        
        loss_test_list.append(loss.item())
    
    print ('Epoch [{}/{}], \t Train accuracy {}%'.format(epoch+1, num_epochs, accuracy_train ))
    print ('Epoch [{}/{}], \t Test accuracy {}%'.format(epoch+1, num_epochs, accuracy_test ))



  0%|                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

  2%|███                                                                                                                                                     | 1/50 [00:09<07:26,  9.11s/it]

Epoch [1/50], 	 Train accuracy 73.385%
Epoch [1/50], 	 Test accuracy 72.49%




  4%|██████                                                                                                                                                  | 2/50 [00:19<07:40,  9.58s/it]

Epoch [2/50], 	 Train accuracy 77.70333333333333%
Epoch [2/50], 	 Test accuracy 76.82%




  6%|█████████                                                                                                                                               | 3/50 [00:30<07:39,  9.78s/it]

...

Epoch [48/50], 	 Train accuracy 90.34%
Epoch [48/50], 	 Test accuracy 87.49%




 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 49/50 [08:30<00:10, 10.65s/it]

Epoch [49/50], 	 Train accuracy 90.35166666666667%
Epoch [49/50], 	 Test accuracy 87.54%




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:40<00:00, 10.41s/it]

Epoch [50/50], 	 Train accuracy 90.51833333333333%
Epoch [50/50], 	 Test accuracy 87.78%


In [195]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= list(range(len(accuracy_train_list))),
    y=accuracy_train_list,
))
fig.add_trace(go.Scatter(
    x= list(range(len(accuracy_test_list))),
    y=accuracy_test_list,

))

fig.show()

In [196]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= list(range(len(loss_train_list))),
    y=loss_train_list,
))
fig.add_trace(go.Scatter(
    x= list(range(len(loss_test_list))),
    y=loss_test_list,

))

fig.show()

### Conclusions:
_Write down small report with your conclusions and your ideas._

1. Удалось натренировать FC-NN на датасете FMNIST с точностью 88.5%

2. Удалось сделать оверфит FC-NN на датасете FMNIST 

3. Удалось побороть оверфит нейросети при помощи механизма Dropout